<a href="https://colab.research.google.com/github/fadoua-bousalim/Covid-19-Data-Viz/blob/main/Covid_Viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU check

---



---



In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')

In [2]:
if gpus : 
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus =  tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), 'Phyiscal GPUs,', len(logical_gpus), 'Logical GPUs')
    except RunTimeError as e :
        print(e)

1 Phyiscal GPUs, 1 Logical GPUs


# Imports & functions

---
---



In [3]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
import seaborn as sns
import time 
import datetime 
import json

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Function to give stats on missing values in a table or dataframe
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [5]:
import datetime 

# String to date funtion
def str_to_date(string) :
  return datetime.datetime.strptime(string, '%Y-%m-%d')

# Difference of two dates
def diff_date(date1, date2) :
  return (date2 - date1).days

# Date parts
def year(string) :
  return str_to_date(string).year

def month_number(string) :
  return str_to_date(string).month

def month_name(string) :
  return str_to_date(string).strftime('%b')

def day(string) :
  return str_to_date(string).day

In [6]:
pip install pycountry_convert &> /dev/null

In [7]:
import pycountry_convert as pc
import pycountry as p

def country(country_code) :
  if country_code == 'XK' :
    country_name = 'Kosovo'
  else :
    # country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
    country_name = p.countries.get(alpha_2=country_code).name
  return country_name

def continent(country_code) :
  if country_code in ('VA', 'SX') :
    continent_name = 'Europe'
  elif country_code == 'TL' :
    continent_name = 'Asia'
  elif country_code == 'PN' :
    continent_name = 'Oceania'
  else :
    continent_code = pc.country_alpha2_to_continent_code(country_code)
    continent_name = pc.convert_continent_code_to_continent_name(continent_code)
  return continent_name

# Data loading & preprocessing

---



---





In [8]:
pip install --upgrade covid19dh &> /dev/null

In [9]:
from covid19dh import covid19
from datetime import date

x, src = covid19(verbose=False, level=1, start=date(2020,1,5), end=date(2021,11,30)) 

In [10]:
x.head()

,id,date,confirmed,deaths,recovered,tests,vaccines,people_vaccinated,people_fully_vaccinated,hosp,icu,vent,school_closing,workplace_closing,cancel_events,gatherings_restrictions,transport_closing,stay_home_restrictions,internal_movement_restrictions,international_movement_restrictions,information_campaigns,testing_policy,contact_tracing,facial_coverings,vaccination_policy,elderly_people_protection,government_response_index,stringency_index,containment_health_index,economic_support_index,administrative_area_level,administrative_area_level_1,administrative_area_level_2,administrative_area_level_3,latitude,longitude,population,iso_alpha_3,iso_alpha_2,iso_numeric,iso_currency,key_local,key_google_mobility,key_apple_mobility,key_jhu_csse,key_nuts,key_gadm
86757,0094b645,2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Saint Lucia,NaN,NaN,13.9094,-60.9789,181889.0,LCA,LC,662.0,XCD,NaN,NaN,NaN,LC,NaN,LCA
86758,0094b645,2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Saint Lucia,NaN,NaN,13.9094,-60.9789,181889.0,LCA,LC,662.0,XCD,NaN,NaN,NaN,LC,NaN,LCA
86759,0094b645,2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Saint Lucia,NaN,NaN,13.9094,-60.9789,181889.0,LCA,LC,662.0,XCD,NaN,NaN,NaN,LC,NaN,LCA
86760,0094b645,2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Saint Lucia,NaN,NaN,13.9094,-60.9789,181889.0,LCA,LC,662.0,XCD,NaN,NaN,NaN,LC,NaN,LCA
86761,0094b645,2020-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Saint Lucia,NaN,NaN,13.9094,-60.9789,181889.0,LCA,LC,662.0,XCD,NaN,NaN,NaN,LC,NaN,LCA


In [11]:
# Columns with more than 56% or missing data
missing_values_table(x)[missing_values_table(x)['% of Total Values']>56].index

Your selected dataframe has 47 columns.
There are 43 columns that have missing values.
Your selected dataframe has 47 columns.
There are 43 columns that have missing values.


Index(['administrative_area_level_2', 'administrative_area_level_3',
       'key_nuts', 'key_local', 'vent', 'icu', 'hosp',
       'people_fully_vaccinated', 'people_vaccinated', 'vaccines',
       'key_apple_mobility', 'tests'],
      dtype='object')

In [12]:
data = x.drop(['administrative_area_level_2', 'administrative_area_level_3',
       'key_nuts', 'key_local', 'vent', 'icu', 'hosp',
       'people_fully_vaccinated', 'people_vaccinated', 'vaccines',
       'key_apple_mobility', 'tests'], axis=1)

In [13]:
data.columns

Index(['id', 'date', 'confirmed', 'deaths', 'recovered', 'school_closing',
       'workplace_closing', 'cancel_events', 'gatherings_restrictions',
       'transport_closing', 'stay_home_restrictions',
       'internal_movement_restrictions', 'international_movement_restrictions',
       'information_campaigns', 'testing_policy', 'contact_tracing',
       'facial_coverings', 'vaccination_policy', 'elderly_people_protection',
       'government_response_index', 'stringency_index',
       'containment_health_index', 'economic_support_index',
       'administrative_area_level', 'administrative_area_level_1', 'latitude',
       'longitude', 'population', 'iso_alpha_3', 'iso_alpha_2', 'iso_numeric',
       'iso_currency', 'key_google_mobility', 'key_jhu_csse', 'key_gadm'],
      dtype='object')

In [14]:
# Handling iso names not handled by pc.country_alpha2_to_continent_code
# for i in data['iso_alpha_2'].unique() :
#   print(i)
#   print(continent(i))

# Removing countries rows with no country code
data = data[data['iso_alpha_2'].notna()]

In [15]:
# missing_values_table(data)

In [16]:
# Keeping only columns with low missing values
data = data[['date', 'confirmed', 'deaths', 'recovered', 'latitude',
       'longitude', 'population', 'iso_alpha_2', 'iso_alpha_3', 'vaccination_policy']]

In [17]:
data.head()

,date,confirmed,deaths,recovered,latitude,longitude,population,iso_alpha_2,iso_alpha_3,vaccination_policy
86757,2020-01-22,NaN,NaN,NaN,13.9094,-60.9789,181889.0,LC,LCA,NaN
86758,2020-01-23,NaN,NaN,NaN,13.9094,-60.9789,181889.0,LC,LCA,NaN
86759,2020-01-24,NaN,NaN,NaN,13.9094,-60.9789,181889.0,LC,LCA,NaN
86760,2020-01-25,NaN,NaN,NaN,13.9094,-60.9789,181889.0,LC,LCA,NaN
86761,2020-01-26,NaN,NaN,NaN,13.9094,-60.9789,181889.0,LC,LCA,NaN


In [18]:
# Creating continent name column
data['continent'] = data['iso_alpha_2'].apply(continent)

In [19]:
# Creating country name columne
data['country'] = data['iso_alpha_2'].apply(country)

In [20]:
# Creating short date 
data['short_date'] = data['date'].dt.strftime("%Y-%m")

In [21]:
data = data[['date', 'short_date', 'continent', 'country', 'iso_alpha_3', 'population', 'latitude', 'longitude', 
             'confirmed', 'deaths', 'recovered', 'vaccination_policy']].reset_index(drop=True)

In [22]:
# Creating continent level agregated data
cont_col = ['date', 'short_date', 'continent', 'population', 
             'confirmed', 'deaths', 'recovered']

continent_data = data[cont_col].groupby(['date', 'short_date', 'continent']).agg({'population':'sum', 'confirmed':'sum', 
             'deaths':'sum', 'recovered':'sum'}).reset_index()

In [23]:
# Add latitude and longitude information for continent (in the center)
continent_data['latitude'] = ""
continent_data['longitude'] = ""

lat_means = data.groupby('continent')['latitude'].mean()
long_means = data.groupby('continent')['longitude'].mean()

lat_means['North America'] = data[data['country']=='Canada']['latitude'].unique()
long_means['North America'] = data[data['country']=='Canada']['longitude'].unique()
lat_means['Oceania'] = data[data['country']=='Australia']['latitude'].unique()
long_means['Oceania'] = data[data['country']=='Australia']['longitude'].unique()

con = continent_data['continent'].unique()

for cont in con :
  continent_data.loc[continent_data.continent==cont, 'latitude'] = lat_means[cont]
  continent_data.loc[continent_data.continent==cont, 'longitude'] = long_means[cont]

In [24]:
# Creating country level agregated data
country_data = data.groupby(['date', 'short_date', 'continent', 'country', 'iso_alpha_3', 'population', 'latitude', 'longitude']).agg({'confirmed':'sum', 
             'deaths':'sum', 'recovered':'sum'}).reset_index()

In [25]:
# Formating floats to integers
for col in ['confirmed', 'deaths', 'recovered'] :
  country_data[col] = country_data[col].astype(int)
  continent_data[col] = continent_data[col].astype(int)

# Simple plots

---
---



In [26]:
pip install plotly &> /dev/null

In [27]:
pip install geoplot &> /dev/null

In [28]:
pip install jupyter-dash -q &> /dev/null

In [46]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Load data
df = country_data

# Define animations
animations = {
    'Scatter': px.scatter(
        df, x="confirmed", y="deaths", animation_frame="short_date", 
        animation_group="country", size="population", color="continent", 
        hover_name="country", log_x=True, 
        range_x=[10,54000000], range_y=[-1000,822000]),
    'Bar': px.bar(
        df, x="continent", y="confirmed", color="continent", 
        animation_frame="short_date", animation_group="country", 
        range_y=[0,continent_data['confirmed'].max()]),
}

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.P("Select an animation:"),
    dcc.RadioItems(
        id='selection',
        options=[{'label': x, 'value': x} for x in animations],
        value='Scatter'
    ),
    dcc.Graph(id="graph"),
])

# Define callback to update graph
@app.callback(
    Output("graph", "figure"), 
    [Input("selection", "value")])

def display_animated_graph(s):
    return animations[s]

# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>

# Geographic plots

---



---



## Data types checks
---



In [30]:
country_data['latitude'] = country_data['latitude'].astype('float')
country_data['longitude'] = country_data['longitude'].astype('float')
country_data['date'] = country_data['date'].astype(str)

## Plots by continent

---



In [31]:
animation = 'short_date'

# Setting animation frame and transition duration depeding on date agregation level (day vs month)
if animation == 'short_date' :
  duration = 1000
else :
  duration = 100

## Plots

---

In [44]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Define animations
animations = {
    'Country': px.scatter_geo(country_data, 
                     lat='latitude', 
                     lon='longitude', 
                     color='continent', 
                     size='confirmed',
                     hover_name='country',
                     projection='equirectangular', 
                     animation_frame='short_date',
                     animation_group='country',
                    #  scope='europe',
                     title='Confirmed cases'),
    'Continent': px.scatter_geo(continent_data, 
                     lat='latitude', 
                     lon='longitude', 
                     color='continent', 
                     size='confirmed',
                     hover_name='continent',
                     projection='equirectangular', 
                     animation_frame='short_date',
                     animation_group='continent',
                    #  scope='europe',
                     title='Confirmed cases'),
}

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.P("Select an animation:"),
    dcc.RadioItems(
        id='selection',
        options=[{'label': x, 'value': x} for x in animations],
        value='Country'
    ),
    dcc.Graph(id="graph"),
])

# Define callback to update graph
@app.callback(
    Output("graph", "figure"), 
    [Input("selection", "value")])

def display_animated_graph(s):
  fig = animations[s]
  fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = duration
  fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = duration/2
  fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["easing"] = 'quadratic-in-out'
  fig.layout.updatemenus[0].buttons[0].label = 'Play'
  fig.layout.updatemenus[0].buttons[1].label = 'Pause'
  fig.update_traces(marker=dict(line=dict(width=1,
                                          color='MediumPurple')))
  fig.update_layout(
      geo=dict(
          showframe=False,
          showcoastlines=False))
  fig.update_layout(margin={"r":0,"t":10,"l":0,"b":0})
  return fig

# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>